# Explorar a Camada Silver

In [1]:
import ipywidgets as widgets
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import dates
#
# Nome da aplicação Spark
#
APP_NAME="ExplorarSilver"

## Inicializar Spark e definir funções de apoio

In [2]:
%run StartSpark.ipynb

## Carregar as tabelas da camada Silver

In [3]:
_ = loadAndRegister("stocks", "StockData", "silver")

In [4]:
%%sql_display
select
    ticker,
    day,
    count(*)
from silver_stocks
group by all
order by ticker, day

,ticker,day,count(1)
0,AAPL,2024-02-07,1
1,AAPL,2024-02-12,143
2,AAPL,2024-02-13,389
3,AAPL,2024-02-14,390
4,AAPL,2024-02-15,401
5,AAPL,2024-02-16,389
6,AMZN,2024-02-12,141
7,AMZN,2024-02-13,388
8,AMZN,2024-02-14,379
9,AMZN,2024-02-15,399


In [5]:
%%sql_display
select
    ticker,
    day,
    timestamp,
    open,
    high,
    low,
    close
from silver_stocks
where ticker="AAPL"
 and day = "2024-02-13"
order by ticker, timestamp
limit 1

,ticker,day,timestamp,open,high,low,close
0,AAPL,2024-02-13,2024-02-13 14:30:00,185.809998,186.190002,185.804993,185.949997


In [6]:
%%sql_display
select
    ticker,
    day,
    timestamp,
    open,
    high,
    low,
    close
from silver_stocks
where ticker="AAPL"
 and day = "2024-02-13"
order by ticker, timestamp desc
limit 1

,ticker,day,timestamp,open,high,low,close
0,AAPL,2024-02-13,2024-02-13 20:59:00,185.089996,185.160004,184.990005,185.070007


In [7]:
%%sql_display
CREATE OR REPLACE TEMPORARY VIEW stocks_data AS 
select * 
from silver_stocks 
order by ticker, day, timestamp

""


In [8]:
%%sql_display
show tables

,namespace,tableName,isTemporary
0,,silver_stocks,False
1,,stocks_data,False


In [9]:
%%sql_display
CREATE OR REPLACE TEMPORARY VIEW gold_daily AS
select
    ticker,
    day,
    first(open) as open,
    min(low) as low,
    max(high) as high,
    last(close) as close,
    sum(volume) as volume
from stocks_data
group by all
order by ticker, day

""


In [10]:
df = %sql select *, (close - open) as osc, (osc / open) as osc_per from gold_daily

In [11]:
(df
  .write
  .format("jdbc")
  .option("url", postgresUrl)
  .option("driver", "org.postgresql.Driver")
  .option("dbtable", "stocks_daily")
  .option("user", postgresUsername)
  .option("password", postgresPassword)
  .mode("overwrite")
  .save()
)

In [12]:
df = %sql \
select \
    _id, \
    ticker, \
    day, \
    description, \
    timestamp - interval 3 hours as timestamp, \
    open, \
    high, \
    low, \
    close, \
    volume, \
    osc, \
    osc_per, \
    __op, \
    __collection, \
    __ts_ms \
from silver_stocks \
order by ticker, timestamp desc

In [13]:
(df
  .write
  .format("jdbc")
  .option("url", postgresUrl)
  .option("driver", "org.postgresql.Driver")
  .option("dbtable", "stocks_intraday")
  .option("user", postgresUsername)
  .option("password", postgresPassword)
  .mode("overwrite")
  .save()
)